# Vote probabilities in Haskell

### Imports

In [1]:
import Statistics.Distribution (probability)
import Statistics.Distribution.Binomial (binomial)

### Per-lovelace probability

In [2]:
-- | Compute the fundamental probability of a lovelace being selected for the committee.
committeeProbabilityFromMeanSize
  :: Int     -- ^ The total lovelace.
  -> Double  -- ^ The mean voting-committee size.
  -> Double  -- ^ The probability of one lovelace being selected for the voting committee.
committeeProbabilityFromMeanSize totalStake meanCommitteeSize = meanCommitteeSize / fromIntegral totalStake

### Committee-membership probabilities

In [3]:
-- | Compute the probability of a party having a given number of votes.
voteProbability
  :: Double  -- ^ Probability of one lovelace being selected for the voting committee.
  -> Int     -- ^ Lovelace held by the party.
  -> Int     -- ^ Number of votes allocated to the party.
  -> Double  -- ^ Probability of the party being allocated the given number of votes.
voteProbability = (probability .) . flip binomial

### Check the mean number of votes

#### All parties

In [4]:
-- | Compute the mean number of votes expected for a party.
meanNumberOfVotes
  :: Double  -- ^ Probability of one lovelace being selected for the voting committee.
  -> Int     -- ^ Lovelace held by the party.
  -> Double  -- ^ Expected number of votes for the party.
meanNumberOfVotes committeeProbability stake = sum [fromIntegral votes * voteProbability committeeProbability stake votes | votes <- [0..stake]]

A test for round-off errors.

In [5]:
testTotalStake = 1000
testCommitteeSize = 200

testP = committeeProbabilityFromMeanSize 1000 200

testMeanVotes = meanNumberOfVotes testP testTotalStake

testMeanVotes

199.9999999999986

In [6]:
abs $ testMeanVotes - testCommitteeSize

1.3926637620897964e-12

#### Several parties

In [7]:
testStakes = [100, 400, 500]

testPartyVotes = meanNumberOfVotes testP <$> testStakes

testPartyVotes

[20.000000000000224,80.0000000000033,100.00000000000468]

In [8]:
sum testPartyVotes

200.00000000000819

A test for round-off errors.

In [9]:
abs $ sum testPartyVotes - testCommitteeSize

8.185452315956354e-12